### Build `PerspectiveTool`

In [ ]:
%pip install google-api-python-client -q

In [ ]:
from googleapiclient import discovery
from typing import Dict, Optional
import json
import os


class Perspective:
    """Custom class to interact with Perspective API."""

    attributes = [
        "toxicity",
        "severe_toxicity",
        "identity_attack",
        "insult",
        "profanity",
        "threat",
        "sexually_explicit",
    ]

    def __init__(self, api_key: Optional[str] = None) -> None:
        if api_key is None:
            try:
                api_key = os.environ["PERSPECTIVE_API_KEY"]
            except KeyError:
                raise ValueError(
                    "Please provide an api key or set PERSPECTIVE_API_KEY env var."
                )

        self._client = discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=api_key,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False,
        )

    def get_toxicity_scores(self, text: str) -> Dict[str, float]:
        """Function that makes API call to Perspective to get toxicity scores across various attributes."""

        analyze_request = {
            "comment": {"text": text},
            "requestedAttributes": {att.upper(): {} for att in self.attributes},
        }

        response = self._client.comments().analyze(body=analyze_request).execute()
        try:
            return {
                att: response["attributeScores"][att.upper()]["summaryScore"]["value"]
                for att in self.attributes
            }
        except Exception as e:
            raise ValueError("Unable to parse response") from e

In [ ]:
perspective = Perspective()

In [ ]:
from typing import Tuple
from llama_index.core.bridge.pydantic import Field


def perspective_function_tool(
    text: str = Field(
        default_factory=str, description="The text to compute toxicity scores on."
    )
) -> Tuple[str, float]:
    """Returns the toxicity score of the most problematic toxic attribute."""

    scores = perspective.get_toxicity_scores(text=text)
    max_key = max(scores, key=scores.get)
    return (max_key, scores[max_key] * 100)


from llama_index.core.tools import FunctionTool

pespective_tool = FunctionTool.from_defaults(
    perspective_function_tool,
)

In [ ]:
perspective_function_tool(text="friendly greetings from python")

('toxicity', 2.5438840000000003)

## Build `InstrospectiveAgentWorker`

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.core.agent.introspective.step import IntrospectiveAgentWorker
from llama_index.core.agent.function_calling.step import FunctionCallingAgentWorker

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

### Define `IntrospectiveAgentWorker`

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgentWorker

main_worker = OpenAIAgentWorker.from_tools(
    tools=[], llm=OpenAI("gpt-4-turbo-preview"), verbose=True
)

In [ ]:
from llama_index.agent.critic import CriticAgentWorker
from llama_index.core.agent import FunctionCallingAgentWorker

critiquer = FunctionCallingAgentWorker.from_tools(
    tools=[pespective_tool], llm=OpenAI("gpt-3.5-turbo")
)

critic = CriticAgentWorker.from_args(
    critique_agent_worker=critiquer,
    critique_template="Please provide a toxicity critique for the following text:\n {input_str}",
)

In [ ]:
introspective_agent = IntrospectiveAgentWorker.from_args(
    main_agent_worker=main_worker,
    reflective_agent_worker=critic,
).as_agent()

In [ ]:
response = introspective_agent.chat(
    "Please provide a more safe version of the following text: {text}".format(
        text="you can barely see the vulva. I tend to holder her hands and wait for"
    )
)

main memory: token_limit=3000 tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all') chat_store=SimpleChatStore(store={}) chat_store_key='chat_history'
Added user message to memory: Please provide a more safe version of the following text: you can barely see the vulva. I tend to holder her hands and wait for
> Running step 8d1bb39f-f134-44ce-b839-6c5a880e96c8. Step input: Please provide a toxicity critique for the following text:
 You can barely see. I tend to hold her hands and wait.
> Running step 24edae56-ae1e-49dc-a2b1-0d9bdc90da7a. Step input: None
Critique: assistant: The toxicity score for the text "You can barely see. I tend to hold her hands and wait." is 3.29, indicating a moderate level of toxicity.
> Running step 84657d7d-290e-42a9-b7fb-76976e909c5e. Step input: Please provide a toxicity critique for the following text:
 You can barely see. I patiently wait, offering my support.
> Running step a31af931-22d2-4ba2-bc0

AttributeError: 'list' object has no attribute 'raw_output'

In [ ]:
response.response

'Here is a corrected version of the input.\nThe visibility is low. I usually wait by staying close.'

In [ ]:
response.sources

[]

In [ ]:
introspective_agent.memory.chat_store.store

{'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, content='Please provide a more safe version of the following text: you can barely see the vulva. I tend to holder her hands and wait for', additional_kwargs={}),
  ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='You can barely see. I tend to hold her hands and wait.', additional_kwargs={}),
  ChatMessage(role=<MessageRole.USER: 'user'>, content='assistant: The toxicity score for the text "You can barely see. I tend to hold her hands and wait." is 3.29, indicating a moderate level of toxicity.', additional_kwargs={}),
  ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Here is a corrected version of the input.\nThe visibility is low. I usually wait by holding her hands.', additional_kwargs={}),
  ChatMessage(role=<MessageRole.USER: 'user'>, content='assistant: The toxicity score of the most problematic toxic attribute in the text "The visibility is low. I usually wait by holding her hands." is 5